In [1]:
import numpy as np

In [2]:
G = {
    "length": 1000
}

In [3]:
#cat map
def cat(starts):
    num = len(starts)
    outputs = np.zeros((num, 2))
    for i in range(num):
        x, y = starts[i][0], starts[i][1]
        output = np.array([(x + y) % 1, (x + 2 * y) % 1])
        outputs[i] += output

    return outputs


In [4]:
#initial conditions
start_pts = np.array(([.1, .5],[.8, .8]))
traj_num = len(start_pts)

In [5]:
#trajectory simulation
def trajectories(starts = start_pts):
    length = G["length"]
    traj = np.zeros((G["length"], traj_num, 2))
    y = starts
    for i in range(length):
        traj[i] += y
        y = cat(y)
    return np.array(traj)
        

In [6]:
#trajectory
Xtrain = trajectories()
#points
points = np.concatenate(Xtrain,0)
#number of cell centers
subset_size = 5

In [7]:
#find cell_centers
def cell_centers(num, pts = points):

    length = len(pts)
    indices = np.random.choice(length, num)
    centers = np.zeros((num, 2))
    for i in range(num):
        index = indices[i]
        #print(index)
        #print(pts[index])
        centers[i] += pts[:][index]
        
    return centers

In [8]:
#cell_centers
sample = cell_centers(subset_size)

#NEIGHBOR SEARCHING
def neighbor_searching(pts = points, centers = sample):
    pt_idxs = np.zeros(len(pts))
    #compute distance
    xs, ys = pts.transpose()[0], pts.transpose()[1]
    x0s, y0s = centers.transpose()[0], centers.transpose()[1]
    for i in range(len(pts)):
        Dxs = np.minimum(abs(xs[i] - x0s), 1 - abs(xs[i] - x0s))
        Dys = np.minimum(abs(ys[i] - y0s), 1 - abs(ys[i] - y0s))
        new_pts = np.array((Dxs, Dys)).transpose()
        distances = np.linalg.norm(new_pts, axis = 1)
        idxs = np.argmin(distances)
        pt_idxs[i] += idxs
        
    return pt_idxs.astype(int)
        
    


In [19]:
#transition step
nt = 1 * traj_num
#contruct Markov matrix using Ulam method
def Ulam_matrix(pts = points):
    M_mat = np.zeros((subset_size, subset_size))
    
    point_idxs = neighbor_searching()
    length = len(points)
    transitions = point_idxs[nt:length] - point_idxs[0:(length - nt)]
    cellEdge = np.flatnonzero(transitions) #+ 1
    
    # Build markov matrix
    M_mat = np.zeros((subset_size, subset_size))
    for ind in range(0, len(cellEdge) - nt):
        ce = cellEdge[ind] 
        M_mat[point_idxs[ce],point_idxs[ce+nt]] += 1.0

    # Normalize rows
    row_sums = np.sum(M_mat, axis=1)
    nonzero_rows = np.nonzero(row_sums)
    row_sums_reshaped = row_sums.reshape(-1, 1)

    M_mat[nonzero_rows] /= row_sums_reshaped


    return M_mat   